In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.rcParams['font.family'] = 'Gulim'

RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

In [2]:
from workalendar.asia import SouthKorea
import pendulum

In [3]:
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')

## 전처리
- 일자에서 월과 일을 분리
- 요일을 레이블 인코딩화(EDA로 요일의 중요도 순 파악)
- 월 별, 일 별 중식 석식 수요 차이 파악

In [4]:
train['월'] = pd.DatetimeIndex(train['일자']).month
test['월'] = pd.DatetimeIndex(test['일자']).month
train['주'] = pd.DatetimeIndex(train['일자']).week
test['주'] = pd.DatetimeIndex(test['일자']).week
train['일'] = pd.DatetimeIndex(train['일자']).day
test['일'] = pd.DatetimeIndex(test['일자']).day

train['출근'] = train['본사정원수']-(train['본사휴가자수']+train['본사출장자수']+train['현본사소속재택근무자수'])
train['휴가비율'] = train['본사휴가자수']/train['본사정원수']
train['출장비율'] = train['본사출장자수']/train['본사정원수']
train['야근비율'] = train['본사시간외근무명령서승인건수']/train['출근']
train['재택비율'] = train['현본사소속재택근무자수']/train['본사정원수']

test['출근'] = test['본사정원수']-(test['본사휴가자수']+test['본사출장자수']+test['현본사소속재택근무자수'])
test['휴가비율'] = test['본사휴가자수']/test['본사정원수']
test['출장비율'] = test['본사출장자수']/test['본사정원수']
test['야근비율'] = test['본사시간외근무명령서승인건수']/test['출근']
test['재택비율'] = test['현본사소속재택근무자수']/test['본사정원수']

train['식사가능자수'] = train['본사정원수'] - train['본사휴가자수'] - train['현본사소속재택근무자수']
test['식사가능자수'] = test['본사정원수'] - test['본사휴가자수'] - test['현본사소속재택근무자수']

# train['중식참여율'] = train['중식계'] / train['식사가능자수']

<ipython-input-4-b11c832c5082>:3: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series.  To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)
  train['주'] = pd.DatetimeIndex(train['일자']).week
<ipython-input-4-b11c832c5082>:4: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series.  To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)
  test['주'] = pd.DatetimeIndex(test['일자']).week


In [5]:
month_rank4dinner = {
    1: 11,
    2: 2,
    3: 1,
    4: 4,
    5: 7,
    6: 6,
    7: 10,
    8: 8,
    9: 5,
    10: 3,
    11: 9,
    12: 12
}
train['월(석식)'] = train['월'].map(month_rank4dinner)
test['월(석식)'] = test['월'].map(month_rank4dinner)

month_rank4lunch = {
    1: 3,
    2: 1,
    3: 2,
    4: 6,
    5: 7,
    6: 8,
    7: 10,
    8: 9,
    9: 5,
    10: 4,
    11: 11,
    12: 12
}
train['월(중식)'] = train['월'].map(month_rank4lunch)
test['월(중식)'] = test['월'].map(month_rank4lunch)

weekday_rank4dinner = {
    '월': 1,
    '화': 2,
    '수': 4,
    '목': 3,
    '금': 5,
}

weekday_rank4lunch = {
    '월': 1,
    '화': 2,
    '수': 3,
    '목': 4,
    '금': 5,
}

train['요일(석식)'] = train['요일'].map(weekday_rank4dinner)
test['요일(석식)'] = test['요일'].map(weekday_rank4dinner)

train['요일(중식)'] = train['요일'].map(weekday_rank4lunch)
test['요일(중식)'] = test['요일'].map(weekday_rank4lunch)

In [6]:
rank = pd.DataFrame(range(1,53))
week_rank_lunch = pd.pivot_table(train,values='중식계',index='주').sort_values(by='중식계').reset_index().drop('중식계',axis=1)
week_rank_dinner = pd.pivot_table(train,values='석식계',index='주').sort_values(by='석식계').reset_index().drop('석식계',axis=1)


week_rank4lunch = {}
for i in range(len(rank)):
    week_rank4lunch[week_rank_lunch['주'][i]] = rank[0][i]


week_rank4dinner = {}
for i in range(len(rank)):
    week_rank4dinner[week_rank_dinner['주'][i]] = rank[0][i]
    
    
train['주(중식)'] = train['주'].map(week_rank4lunch)
test['주(중식)'] = test['주'].map(week_rank4lunch)

train['주(석식)'] = train['주'].map(week_rank4dinner)
test['주(석식)'] = test['주'].map(week_rank4dinner)

In [7]:
def is_holiday(date):
    holidays = list(map(str, pd.Series(np.array(SouthKorea().holidays(int(date[:4])))[:, 0])))
    
    yesterday = str(np.datetime64(date) - 1)
    tomorrow = str(np.datetime64(date) + 1)

    if tomorrow in holidays and yesterday in holidays:
        return 3
    if tomorrow in holidays:
        return 2
    elif yesterday in holidays:
        return 1
    else : 
        return 0

def week_of_month(x):
    dt = pendulum.parse(x)
    
    wom = dt.week_of_month
    if wom < 0:
        wom += 52
    return wom
    

df = pd.concat([train[['본사정원수', '일자']], test[['본사정원수', '일자']]])
df['년월'] = df['일자'].apply(lambda x : x[:7])
df = df[['년월', '본사정원수']].groupby(by=['년월'], as_index=False).mean()

def member_change(date):
    this_month = date[:7]
    last_month = str(np.datetime64(this_month) - 1)
    
    this_month_member = int(df[df['년월'] == this_month]['본사정원수'])
    last_month_member = int(df[df['년월'] == last_month]['본사정원수'])
    
    
    return  this_month_member - last_month_member

train['공휴일전후'] = train['일자'].apply(is_holiday)
test['공휴일전후'] = test['일자'].apply(is_holiday)

train['몇주차'] = train['일자'].apply(week_of_month)
test['몇주차'] = test['일자'].apply(week_of_month)

train = train[train['일자'] > '2016-03']
train['인원변화'] = train['일자'].apply(member_change)
test['인원변화'] = test['일자'].apply(member_change)

## 공휴일 변수 생성

In [8]:
train.columns

Index(['일자', '요일', '본사정원수', '본사휴가자수', '본사출장자수', '본사시간외근무명령서승인건수',
       '현본사소속재택근무자수', '조식메뉴', '중식메뉴', '석식메뉴', '중식계', '석식계', '월', '주', '일',
       '출근', '휴가비율', '출장비율', '야근비율', '재택비율', '식사가능자수', '월(석식)', '월(중식)',
       '요일(석식)', '요일(중식)', '주(중식)', '주(석식)', '공휴일전후', '몇주차', '인원변화'],
      dtype='object')

In [9]:
# 메뉴 변수 없이 사용할떄 해당 코드 사용['공휴일전후', '몇주차', '인원변화']

lunch_train = train[['공휴일전후', '몇주차', '인원변화', '요일(중식)','월(중식)','일','주(중식)','출근', '휴가비율', '출장비율', '야근비율', '재택비율','본사출장자수','본사휴가자수','식사가능자수','본사시간외근무명령서승인건수']]
lunch_test = test[['공휴일전후', '몇주차', '인원변화', '요일(중식)','월(중식)','일','주(중식)','출근', '휴가비율', '출장비율', '야근비율', '재택비율','본사출장자수','본사휴가자수','식사가능자수','본사시간외근무명령서승인건수']]

dinner_train= train[['공휴일전후', '몇주차', '인원변화', '요일(석식)','월(석식)','일','주(석식)','출근', '휴가비율', '출장비율', '야근비율', '재택비율','본사출장자수','본사휴가자수','식사가능자수','본사시간외근무명령서승인건수']]
dinner_test = test[['공휴일전후', '몇주차', '인원변화', '요일(석식)','월(석식)','일','주(석식)','출근', '휴가비율', '출장비율', '야근비율', '재택비율','본사출장자수','본사휴가자수','식사가능자수','본사시간외근무명령서승인건수']]

In [10]:
lunch_train.columns

Index(['공휴일전후', '몇주차', '인원변화', '요일(중식)', '월(중식)', '일', '주(중식)', '출근', '휴가비율',
       '출장비율', '야근비율', '재택비율', '본사출장자수', '본사휴가자수', '식사가능자수', '본사시간외근무명령서승인건수'],
      dtype='object')

In [11]:
print(lunch_train.shape)
print(lunch_test.shape)

(1187, 16)
(50, 16)


In [12]:
print(dinner_train.shape)
print(dinner_test.shape)

(1187, 16)
(50, 16)


In [13]:
cat_features = [f for f in lunch_train.columns if lunch_train[f].dtype == 'object']

def column_index(df, cat_features):
    cols = df.columns.values
    sidx = np.argsort(cols)
    return sidx[np.searchsorted(cols, cat_features, sorter=sidx)]

cat_features_idx = column_index(lunch_train, cat_features)    
print("Cat features are: %s" % [f for f in cat_features])
print(cat_features_idx)

Cat features are: []
[]


In [14]:
y_lunch = train[['중식계']]
y_dinner = train[['석식계']]

#### 분포 확인 및 분포 조정

# 중식 예측모델

In [15]:
from automl_alex import LightGBMRegressor, BestSingleModel
from sklearn.metrics import mean_absolute_error as MAE
import sklearn
import time
import os
import contextlib
import shutil

In [16]:
params = {
            "clean_and_encod_data": True,
            "opt_data_prepare": True,
            "metric": MAE,
            "metric_round": 4,
            "auto_parameters": False,
            "folds": 8,
            "score_folds": 3,
            "opt_lvl": 3,
            "early_stoping": 120,
            "feature_selection": False,
            "type_of_estimator": "regression",
            "random_state": 42,
            "gpu": False,
            "cat_encoder_names": [
                "HelmertEncoder",
                "OneHotEncoder",
                "CountEncoder",
                "HashingEncoder",
                "BackwardDifferenceEncoder",
            ],
            "target_encoders_names": [
                "TargetEncoder",
                "JamesSteinEncoder",
                "CatBoostEncoder",
            ],
            "clean_outliers": [True, False],
            "num_generator_select_operations": True,
            "num_generator_operations": ["/", "*", "-", "+"],
        }

In [ ]:
lunch_model = BestSingleModel(
            models_names=[
                "LightGBM",
            ],
            **params,
        )


lunch_history = lunch_model.opt(
            X=lunch_train,
            y=y_lunch,
            timeout=1100,
            verbose=3,
            fit_end=False,
        )

22:48:26 | ##################################################
--- Logging error in Loguru Handler #4 ---
Record was: {'elapsed': datetime.timedelta(microseconds=840009), 'exception': None, 'extra': {}, 'file': (name='optimizer.py', path='C:\\anaconda3\\envs\\machinelearning\\lib\\site-packages\\automl_alex\\optimizer.py'), 'function': 'opt', 'level': (name='INFO', no=20, icon='ℹ️'), 'line': 557, 'message': '##################################################', 'module': 'optimizer', 'name': 'automl_alex.optimizer', 'process': (id=10284, name='MainProcess'), 'thread': (id=13036, name='MainThread'), 'time': datetime(2021, 7, 15, 22, 48, 26, 906819, tzinfo=datetime.timezone(datetime.timedelta(seconds=32400), '대한민국 표준시'))}
Traceback (most recent call last):
  File "C:\anaconda3\envs\machinelearning\lib\site-packages\loguru\_handler.py", line 177, in emit
    self._sink.write(str_record)
  File "C:\anaconda3\envs\machinelearning\lib\site-packages\loguru\_file_sink.py", line 174, in write
   

--- Logging error in Loguru Handler #10 ---
Record was: {'elapsed': datetime.timedelta(seconds=4, microseconds=149161), 'exception': None, 'extra': {}, 'file': (name='data_prepare.py', path='C:\\anaconda3\\envs\\machinelearning\\lib\\site-packages\\automl_alex\\data_prepare.py'), 'function': 'fit_transform', 'level': (name='INFO', no=20, icon='ℹ️'), 'line': 1067, 'message': ' ADD features: 360', 'module': 'data_prepare', 'name': 'automl_alex.data_prepare', 'process': (id=10284, name='MainProcess'), 'thread': (id=13036, name='MainThread'), 'time': datetime(2021, 7, 15, 22, 48, 30, 215971, tzinfo=datetime.timezone(datetime.timedelta(seconds=32400), '대한민국 표준시'))}
Traceback (most recent call last):
  File "C:\anaconda3\envs\machinelearning\lib\site-packages\loguru\_handler.py", line 177, in emit
    self._sink.write(str_record)
  File "C:\anaconda3\envs\machinelearning\lib\site-packages\loguru\_file_sink.py", line 174, in write
    self._terminate_file(is_rotating=True)
  File "C:\anaconda

--- Logging error in Loguru Handler #16 ---
Record was: {'elapsed': datetime.timedelta(seconds=5, microseconds=921034), 'exception': None, 'extra': {}, 'file': (name='data_prepare.py', path='C:\\anaconda3\\envs\\machinelearning\\lib\\site-packages\\automl_alex\\data_prepare.py'), 'function': 'fit_transform', 'level': (name='INFO', no=20, icon='ℹ️'), 'line': 924, 'message': 'Source data shape: (1187, 16)', 'module': 'data_prepare', 'name': 'automl_alex.data_prepare', 'process': (id=10284, name='MainProcess'), 'thread': (id=13036, name='MainThread'), 'time': datetime(2021, 7, 15, 22, 48, 31, 987844, tzinfo=datetime.timezone(datetime.timedelta(seconds=32400), '대한민국 표준시'))}
Traceback (most recent call last):
  File "C:\anaconda3\envs\machinelearning\lib\site-packages\loguru\_handler.py", line 177, in emit
    self._sink.write(str_record)
  File "C:\anaconda3\envs\machinelearning\lib\site-packages\loguru\_file_sink.py", line 174, in write
    self._terminate_file(is_rotating=True)
  File "C

--- Logging error in Loguru Handler #16 ---
Record was: {'elapsed': datetime.timedelta(seconds=6, microseconds=165835), 'exception': None, 'extra': {}, 'file': (name='data_prepare.py', path='C:\\anaconda3\\envs\\machinelearning\\lib\\site-packages\\automl_alex\\data_prepare.py'), 'function': 'fit_transform', 'level': (name='INFO', no=20, icon='ℹ️'), 'line': 1067, 'message': ' ADD features: 360', 'module': 'data_prepare', 'name': 'automl_alex.data_prepare', 'process': (id=10284, name='MainProcess'), 'thread': (id=13036, name='MainThread'), 'time': datetime(2021, 7, 15, 22, 48, 32, 232645, tzinfo=datetime.timezone(datetime.timedelta(seconds=32400), '대한민국 표준시'))}
Traceback (most recent call last):
  File "C:\anaconda3\envs\machinelearning\lib\site-packages\loguru\_handler.py", line 177, in emit
    self._sink.write(str_record)
  File "C:\anaconda3\envs\machinelearning\lib\site-packages\loguru\_file_sink.py", line 174, in write
    self._terminate_file(is_rotating=True)
  File "C:\anaconda

--- Logging error in Loguru Handler #22 ---
Record was: {'elapsed': datetime.timedelta(seconds=9, microseconds=98261), 'exception': None, 'extra': {}, 'file': (name='data_prepare.py', path='C:\\anaconda3\\envs\\machinelearning\\lib\\site-packages\\automl_alex\\data_prepare.py'), 'function': 'fit_transform', 'level': (name='INFO', no=20, icon='ℹ️'), 'line': 924, 'message': 'Source data shape: (1187, 16)', 'module': 'data_prepare', 'name': 'automl_alex.data_prepare', 'process': (id=10284, name='MainProcess'), 'thread': (id=13036, name='MainThread'), 'time': datetime(2021, 7, 15, 22, 48, 35, 165071, tzinfo=datetime.timezone(datetime.timedelta(seconds=32400), '대한민국 표준시'))}
Traceback (most recent call last):
  File "C:\anaconda3\envs\machinelearning\lib\site-packages\loguru\_handler.py", line 177, in emit
    self._sink.write(str_record)
  File "C:\anaconda3\envs\machinelearning\lib\site-packages\loguru\_file_sink.py", line 174, in write
    self._terminate_file(is_rotating=True)
  File "C:

...

--- Logging error in Loguru Handler #76 ---
Record was: {'elapsed': datetime.timedelta(seconds=49, microseconds=683481), 'exception': None, 'extra': {}, 'file': (name='data_prepare.py', path='C:\\anaconda3\\envs\\machinelearning\\lib\\site-packages\\automl_alex\\data_prepare.py'), 'function': 'fit_transform', 'level': (name='INFO', no=20, icon='ℹ️'), 'line': 1067, 'message': ' ADD features: 240', 'module': 'data_prepare', 'name': 'automl_alex.data_prepare', 'process': (id=10284, name='MainProcess'), 'thread': (id=13036, name='MainThread'), 'time': datetime(2021, 7, 15, 22, 49, 15, 750291, tzinfo=datetime.timezone(datetime.timedelta(seconds=32400), '대한민국 표준시'))}
Traceback (most recent call last):
  File "C:\anaconda3\envs\machinelearning\lib\site-packages\loguru\_handler.py", line 177, in emit
    self._sink.write(str_record)
  File "C:\anaconda3\envs\machinelearning\lib\site-packages\loguru\_file_sink.py", line 174, in write
    self._terminate_file(is_rotating=True)
  File "C:\anacond

Optimize: : 2it [00:09,  5.38s/it, | Model: LightGBM | OptScore: 70.2406 | Best mean_absolute_error: 68.5715 ]--- Logging error in Loguru Handler #82 ---
Record was: {'elapsed': datetime.timedelta(seconds=56, microseconds=712893), 'exception': None, 'extra': {}, 'file': (name='data_prepare.py', path='C:\\anaconda3\\envs\\machinelearning\\lib\\site-packages\\automl_alex\\data_prepare.py'), 'function': 'fit_transform', 'level': (name='INFO', no=20, icon='ℹ️'), 'line': 924, 'message': 'Source data shape: (1187, 16)', 'module': 'data_prepare', 'name': 'automl_alex.data_prepare', 'process': (id=10284, name='MainProcess'), 'thread': (id=13036, name='MainThread'), 'time': datetime(2021, 7, 15, 22, 49, 22, 779703, tzinfo=datetime.timezone(datetime.timedelta(seconds=32400), '대한민국 표준시'))}
Traceback (most recent call last):
  File "C:\anaconda3\envs\machinelearning\lib\site-packages\loguru\_handler.py", line 177, in emit
    self._sink.write(str_record)
  File "C:\anaconda3\envs\machinelearning\l

--- Logging error in Loguru Handler #82 ---
Record was: {'elapsed': datetime.timedelta(seconds=57, microseconds=1559), 'exception': None, 'extra': {}, 'file': (name='data_prepare.py', path='C:\\anaconda3\\envs\\machinelearning\\lib\\site-packages\\automl_alex\\data_prepare.py'), 'function': 'fit_transform', 'level': (name='INFO', no=20, icon='ℹ️'), 'line': 1067, 'message': ' ADD features: 360', 'module': 'data_prepare', 'name': 'automl_alex.data_prepare', 'process': (id=10284, name='MainProcess'), 'thread': (id=13036, name='MainThread'), 'time': datetime(2021, 7, 15, 22, 49, 23, 68369, tzinfo=datetime.timezone(datetime.timedelta(seconds=32400), '대한민국 표준시'))}
Traceback (most recent call last):
  File "C:\anaconda3\envs\machinelearning\lib\site-packages\loguru\_handler.py", line 177, in emit
    self._sink.write(str_record)
  File "C:\anaconda3\envs\machinelearning\lib\site-packages\loguru\_file_sink.py", line 174, in write
    self._terminate_file(is_rotating=True)
  File "C:\anaconda3\

Optimize: : 3it [00:14,  5.22s/it, | Model: LightGBM | OptScore: 72.3145 | Best mean_absolute_error: 68.5715 ]--- Logging error in Loguru Handler #88 ---
Record was: {'elapsed': datetime.timedelta(seconds=61, microseconds=902627), 'exception': None, 'extra': {}, 'file': (name='data_prepare.py', path='C:\\anaconda3\\envs\\machinelearning\\lib\\site-packages\\automl_alex\\data_prepare.py'), 'function': 'fit_transform', 'level': (name='INFO', no=20, icon='ℹ️'), 'line': 924, 'message': 'Source data shape: (1187, 16)', 'module': 'data_prepare', 'name': 'automl_alex.data_prepare', 'process': (id=10284, name='MainProcess'), 'thread': (id=13036, name='MainThread'), 'time': datetime(2021, 7, 15, 22, 49, 27, 969437, tzinfo=datetime.timezone(datetime.timedelta(seconds=32400), '대한민국 표준시'))}
Traceback (most recent call last):
  File "C:\anaconda3\envs\machinelearning\lib\site-packages\loguru\_handler.py", line 177, in emit
    self._sink.write(str_record)
  File "C:\anaconda3\envs\machinelearning\l

Traceback (most recent call last):
  File "C:\anaconda3\envs\machinelearning\lib\site-packages\loguru\_handler.py", line 177, in emit
    self._sink.write(str_record)
  File "C:\anaconda3\envs\machinelearning\lib\site-packages\loguru\_file_sink.py", line 174, in write
    self._terminate_file(is_rotating=True)
  File "C:\anaconda3\envs\machinelearning\lib\site-packages\loguru\_file_sink.py", line 205, in _terminate_file
    os.rename(old_path, renamed_path)
PermissionError: [WinError 32] 다른 프로세스가 파일을 사용 중이기 때문에 프로세스가 액세스 할 수 없습니다: 'C:\\Users\\김영봉\\Desktop\\Project\\DACON\\식수인원예측\\notebook\\.automl-alex_tmp\\log.log' -> 'C:\\Users\\김영봉\\Desktop\\Project\\DACON\\식수인원예측\\notebook\\.automl-alex_tmp\\log.2021-07-15_16-29-06_270718.log'
--- End of logging error ---
--- Logging error in Loguru Handler #88 ---
Record was: {'elapsed': datetime.timedelta(seconds=62, microseconds=283961), 'exception': None, 'extra': {}, 'file': (name='data_prepare.py', path='C:\\anaconda3\\envs\\machinelearning\\

In [ ]:
if lunch_model.direction == "maximize":
    top_10_cfg = history.sort_values("value", ascending=False).head(5)
else:
    top_10_cfg = history.sort_values("value", ascending=True).head(5)

_tmp_models_folder = "../autoML_models/"
if os.path.isdir(_tmp_models_folder):
    shutil.rmtree(_tmp_models_folder)

with open(os.devnull, "w") as f, contextlib.redirect_stdout(f):
    for i in range(5):
        n_model = top_10_cfg.number.values[i]
        model = lunch_model.get_model_from_iter(
            lunch_train, y_lunch, lunch_model.study.trials[n_model].params
        )
        model.save(f"model_{i+1}", folder=_tmp_models_folder)

In [ ]:
models_predicts = []
with open(os.devnull, "w") as f, contextlib.redirect_stdout(f):
    for i in range(5):
        model = lunch_model.load(
            f"model_{i+1}", folder=_tmp_models_folder, verbose=0
        )
        predicts = model.predict(lunch_test)
        models_predicts.append(predicts)

lunch_predict = pd.DataFrame(models_predicts).mean()

In [ ]:
dinner_model = BestSingleModel(
            models_names=[
                "LightGBM",
            ],
            **params,
        )


dinner_history = dinner_model.opt(
                                  X=dinner_train,
                                  y=y_dinner,
                                  timeout=1100,
                                  verbose=3,
                                  fit_end=False,
                                 )

In [ ]:
if dinner_model.direction == "maximize":
    top_10_cfg = dinner_history.sort_values("value", ascending=False).head(5)
else:
    top_10_cfg = dinner_history.sort_values("value", ascending=True).head(5)

_tmp_models_folder = "../autoML_models/"
if os.path.isdir(_tmp_models_folder):
    shutil.rmtree(_tmp_models_folder)

with open(os.devnull, "w") as f, contextlib.redirect_stdout(f):
    for i in range(5):
        n_model = top_10_cfg.number.values[i]
        model = dinner_model.get_model_from_iter(
            dinner_train, y_dinner, dinner_model.study.trials[n_model].params
        )
        model.save(f"model_{i+1}", folder=_tmp_models_folder)

In [ ]:
models_predicts = []
with open(os.devnull, "w") as f, contextlib.redirect_stdout(f):
    for i in range(5):
        model = dinner_model.load(
            f"model_{i+1}", folder=_tmp_models_folder, verbose=0
        )
        predicts = model.predict(dinner_test)
        models_predicts.append(predicts)

dinner_predict = pd.DataFrame(models_predicts).mean()

In [ ]:
submission = pd.read_csv('../data/sample_submission.csv')

In [ ]:
submission.iloc[:,1] = lunch_predict
submission.iloc[:,2] = dinner_predict
submission.head()

In [ ]:
answer = pd.read_csv('../submission/20210715_lgbm_autoML.csv')

lunch_answer = np.array(answer.iloc[:,1])
dinner_answer = np.array(answer.iloc[:,2])

abs(lunch_model - lunch_answer).mean(), abs(dinner_model - dinner_answer).mean()

# 저장

In [ ]:
import datetime
today = str(datetime.datetime.now().date()).replace("-","")
print("오늘 날짜 : " + today)

submission.to_csv(f'../submission/{today}_lgbm_sort_autoML.csv', index =False)